# CS231n Winter 2016: Lecture 7
## Topics
- CNN
**TODO:** add topics. hm maybe there is some plugin to do this quick?

## Sources
- video: https://www.youtube.com/watch?v=LxfUGhug-iQ
- original notes by Andrej Karpathy: 

In [1]:
from IPython.display import HTML
video_id='LxfUGhug-iQ'
HTML(f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

In [2]:
import numpy as np

### Convolution filters
- filter == kernal
- examples:
Roberts filter(2x2), Sobel filter(3x3), Prewitt filter(3x3), Laplacian filter(3x3), Gauss-Laplacian filter(5x5), Krisch filter(3x3).

## Convolution layer
- convolute (slide) over all spatical locations
- result - activation map
- apply ReLU
- params: `width, height, depth, stride, pad`
```python
N = input.width
F = filter.width
P = padding
S = stride
output_width = (N - F + 2*P) / S + 1
```
- weights
```python
one_filter = N*N*input_depth + 1 # +1 bies
one_layer_filters = one_filter*num_of_filters_in_one_layer
```

## Visualization of layers
- [Visualizing and Understanding Convolutional Networks by Matthew D Zeiler, Rob Fergus](https://arxiv.org/abs/1311.2901)
- **TODO:** try to visualize
  - filters
  - activation maps

### Padding
- prevent shrinking of activation map 
- **TODO**: doesn't padding introduce borders? We could have almost white image and `0` padding will add border which could activate filter which detects edges and in this way all our images would have encoded borders.
_Actually `0` doesn't contribute to the product of filter to input values and actually `0` is not black it is 'grey'. So it seem that edge filter will get the same result as just squre filter, or line filter. So padded number won't contribute to the activation map at all._
- **TODO**: btw doesn't it mean that edges on grey background (input `0`) won't be seen by edge filter?

### 1x1 convolution layer
decreases number activation map depth

## Pooling layer
- downsampeling
- maxpooling
- avgpooling
- params: filter_size (F), stride (S). Usually: `F=2, S=2` and `F=3, S=2`
- trend toward getting rid of POOL/FC layers (just CONV with strides)

## Estimation (ILSVRC)
- top 5 error - because a lot of categories 1000 we give 5 chances to guess what is the single image is

## Case Study: LeNet-5 (1998)

## Just Before AlexNet
- `28.2% (ILSVRC'10) -> 25.8% (ILSVRC'11) (top 5 error)`
- layers: shallow 
- in test time could be run on phone

## Case Study: AlexNet (2012)
- 18.2% -> 15.4%
- `16.4% (top 5 error)`
- layers: 8
- 2 separated streams - because they have 2 GPUs - which is not problem right now and we don't need 2 streams
- normalization layer (NORM) - which we don't use any more
- 1st use of ReLU
- heavy data augmentation
- dropout 0.5
- batch size 128
- SGD Momentum 0.9
- learning rate 1e-2, reduced by 10 manually when val accuracy plateaus
- L2 weight decay 5e-4
- 7 CNN ensemble: 18.2% -> 15.4%
- FC6, FC7, FC9 - fully connected layers
  - sometimes people refer to `FC7` - as the last layer before classifer

## Case Study: ZFNet (2013)
- 15.4% -> 14.8%
- `11.7% (top 5 error)`
- layers: 8
- CONV1: 11x11 stride 4 -> 7x7 stride 2
- CONV3,4,5: depth 384, 384, 256 -> 512, 1024, 512

## Case Study: VGGNet (2014)
- `7.3% (top 5 error)`
- layers: 19
- footprint of the callculations: 24M * 4bytes ~ 93Mb (weights from forward propagation)
- parameters: 138M
- only 3x3 CONV stride 1, poll 1
- and 2x2 MAX POOL stride 2
- TODO: could get actual architecture [here](https://youtu.be/LxfUGhug-iQ?list=PLkt2uSq6rBVctENoVBg1TpCC7OQi31AlC&t=3535)

problem:
- a lot of parameters on 1st FC but the most data on the 1st few CNN layers. Because it connected to the previous activation map
  - solution: use POOLAVG and shrink 7x7x512 -> 1x1x512

## Case Study: Google Net (2014)
- `6.7%` (top 5 error)
- layers: 22
- inception module (many parallel CONV
- only 5M parameters (remove FC layers) - 12x less parameters

## Case Study: ResNet (2015)
- `3.57% (top 5 error)`
- layers: 152
- Microsoft
- 2-3 weeks on 8 GPU
- faster than VGG (?)
- decrease spatial resolution very early (by MAX POLL) - what decrease number of params and gives posibility to increase number of layers
- Batch Normalization for every CONV layer
- Xavier/2 initialization
- SGD + Momentum (0.9)
- Learning rate: 0.1, divided by 10 when validation error plateaus
- Mini-batch size 256
- Weight decay of 1e-5
- No dropout used (because they told that with batch norm you don't need it)
- additional subtleties https://youtu.be/LxfUGhug-iQ?list=PLkt2uSq6rBVctENoVBg1TpCC7OQi31AlC&t=4480
  - all 3x3
  - bottleneck (for ResNET-50/101/152)

## Case Study: ? (2016)
TODO: research

## Case Study: ? (2017)
TODO: research

## Case Study: ? (2018)
TODO: research

## Case Study: CIFAR-10 experiments
- CIFAR-10 plain net
- CIFAR-10 ResNets
- Residual Net - have skip connections
  - add identity before last ReLU
  - 2 skipped layers compute delta of your representation (classifer gradient) instead of new representation

## Case Study: Deep Mind's (AlphaGo)
- Input: [19x19x48], because 48 feature planes of each position
- CONV1: 192 5x5 filters, stride 1, pad 2 => 19x19x192
- CONV2..12: 192 3x3 filters, stride 1, pad => 19x19x192
- CONV: 1x1 filter, stride 1, pad 0 (with different bies for each position), softmax => [19x19] (probability map of promising moves)

## Summary
- typical architecture: 
`[(CONV-RELU)*N-POLL?]*M-(FC-RELU)*K,SOFTMAX`
  - N ~ 5
  - M is large
  - K <= 2